In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from models import *
from trainers import *
from eval_tools.lossfuncs import NTXentLoss
from data_utils import *

In [2]:
device = 'cuda:1'

model = MoCo(base_model='resnet10', out_dim=128, K=128, from_small=True)
dataloaders, dataset_sizes = data_loader('cifar100', 
                                         './data/cifar100_moco', 
                                         rep_augment='moco',
                                         batch_size=128,
                                         valid_size=0)

optimizer = optim.SGD(model.parameters(), lr=0.3, momentum=0.9, weight_decay=1e-4)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=80, eta_min=0.00001)

criterion = NTXentLoss(device, 
                       batch_size=128, 
                       temperature=0.5, 
                       use_cosine_similarity=True)

Feature extractor: resnet10
Feature extractor: resnet10
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainer = MoCoTrainer(model, 
                      dataloaders, 
                      dataset_sizes, 
                      criterion, 
                      optimizer, 
                      scheduler, 
                      device)

In [4]:
trainer.train(num_epochs=80)

Train start on device: GeForce RTX 2080 Ti

[Epoch 1/80] Elapsed 15.918s/it
[Train] Loss - 0.0361, Accuracy - 4.61 Learning Rate - 0.299884
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 2/80] Elapsed 13.352s/it
[Train] Loss - 0.0337, Accuracy - 8.54 Learning Rate - 0.299538
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 3/80] Elapsed 13.255s/it
[Train] Loss - 0.0325, Accuracy - 11.60 Learning Rate - 0.298960
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 4/80] Elapsed 13.162s/it
[Train] Loss - 0.0320, Accuracy - 13.46 Learning Rate - 0.298153
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 5/80] Elapsed 13.129s/it
[Train] Loss - 0.0318, Accuracy - 14.76 Learning Rate - 0.297118
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 6/80] Elapsed 13.427s/it
[Train] Loss - 0.0317, Accuracy - 15.18 Learning Rate - 0.295856
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 7/80] Elapsed 13.398s/it
[Train] Loss - 0.0315, Accuracy - 15.84 Learning Rate - 0.294368
[Valid] Loss - 0.0000, Accuracy - 0.00 

[Epoch 44/80] Elapsed 13.661s/it
[Train] Loss - 0.0310, Accuracy - 19.48 Learning Rate - 0.126541
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 45/80] Elapsed 13.854s/it
[Train] Loss - 0.0309, Accuracy - 19.63 Learning Rate - 0.120742
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 46/80] Elapsed 13.285s/it
[Train] Loss - 0.0309, Accuracy - 19.53 Learning Rate - 0.114989
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 47/80] Elapsed 13.479s/it
[Train] Loss - 0.0309, Accuracy - 19.91 Learning Rate - 0.109290
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 48/80] Elapsed 13.202s/it
[Train] Loss - 0.0309, Accuracy - 19.66 Learning Rate - 0.103654
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 49/80] Elapsed 13.639s/it
[Train] Loss - 0.0309, Accuracy - 19.96 Learning Rate - 0.098089
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 50/80] Elapsed 13.717s/it
[Train] Loss - 0.0309, Accuracy - 20.08 Learning Rate - 0.092604
[Valid] Loss - 0.0000, Accuracy - 0.00 
[Epoch 51/80] Elapsed 13.23s/it
[T

In [5]:
probe_loaders, probe_sizes = data_loader('cifar100', './data/cifar100_valid5000', rep_augment=None, batch_size=128, valid_size=5000)

probe_setup = {
    'dataloaders': probe_loaders,
    'dataset_sizes': probe_sizes,
    'num_classes': 100
}

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [6]:
trainer.test('./results/MoCo', do_probe=True, probe_setup=probe_setup)

[Test] Loss - 0.0305, Measure - 23.09%
========= Linear Probe Training =========
[Linear Probe Epoch 1/20]
[Train] Loss - 4.7997, Top1 Acc - 5.88% 
[Valid] Loss - 4.3789, Top1 Acc - 9.66% 
[Valid] Top5 Acc - 28.74% 
---------------------------------------------
[Linear Probe Epoch 2/20]
[Train] Loss - 4.4062, Top1 Acc - 9.59% 
[Valid] Loss - 4.0470, Top1 Acc - 11.80% 
[Valid] Top5 Acc - 31.12% 
---------------------------------------------
[Linear Probe Epoch 3/20]
[Train] Loss - 4.1611, Top1 Acc - 11.82% 
[Valid] Loss - 3.9171, Top1 Acc - 12.48% 
[Valid] Top5 Acc - 32.52% 
---------------------------------------------
[Linear Probe Epoch 4/20]
[Train] Loss - 4.0977, Top1 Acc - 13.06% 
[Valid] Loss - 3.9787, Top1 Acc - 13.34% 
[Valid] Top5 Acc - 33.26% 
---------------------------------------------
[Linear Probe Epoch 5/20]
[Train] Loss - 3.9738, Top1 Acc - 14.11% 
[Valid] Loss - 3.9092, Top1 Acc - 13.56% 
[Valid] Top5 Acc - 33.52% 
---------------------------------------------
[Linear